In [1]:
"""
Asynchronous Advantage Actor Critic (A3C) with continuous action space, Reinforcement Learning.

The Pendulum example.

View more on my tutorial page: https://morvanzhou.github.io/tutorials/

Using:
tensorflow 1.0
gym 0.8.0
"""

import multiprocessing
import threading
import tensorflow as tf
import numpy as np
import gym
import os
import shutil
import matplotlib.pyplot as plt

GAME = 'BipedalWalker-v2'
#GAME = 'Pendulum-v0'
OUTPUT_GRAPH = True
LOG_DIR = './log'
N_WORKERS = 2#multiprocessing.cpu_count()
MAX_EP_STEP = 400
MAX_GLOBAL_EP = 11000
GLOBAL_NET_SCOPE = 'Global_Net'
UPDATE_GLOBAL_ITER = 20
GAMMA = 0.95
ENTROPY_BETA = 0.1
LR_A = 0.00005    # learning rate for actor
LR_C = 0.01    # learning rate for critic
GLOBAL_RUNNING_R = []
GLOBAL_EP = 0
Train = True
Train = False
CHECKPOINT_SAVE_PATH = "/tmp/"+"a3c_NN_sum_r10"+".ckpt"
CHECKPOINT_INTERVAL = 500


env = gym.make(GAME)

N_S = env.observation_space.shape[0]
N_A = env.action_space.shape[0]
A_BOUND = [env.action_space.low, env.action_space.high]


class ACNet(object):
    def __init__(self, scope, globalAC=None):

        if scope == GLOBAL_NET_SCOPE:   # get global network
            with tf.variable_scope(scope):
                self.s = tf.placeholder(tf.float32, [None, N_S], 'S')
                self._build_net()
                self.a_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/actor')
                self.c_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/critic')
        else:   # local net, calculate losses
            with tf.variable_scope(scope):
                self.s = tf.placeholder(tf.float32, [None, N_S], 'S')
                self.a_his = tf.placeholder(tf.float32, [None, N_A], 'A')
                self.v_target = tf.placeholder(tf.float32, [None, 1], 'Vtarget')

                mu, sigma, self.v = self._build_net()

                td = tf.subtract(self.v_target, self.v, name='TD_error')
                with tf.name_scope('c_loss'):
                    self.c_loss = tf.reduce_mean(tf.square(td))

                with tf.name_scope('wrap_a_out'):
                    mu, sigma = mu * A_BOUND[1], sigma + 1e-10

                normal_dist = tf.contrib.distributions.Normal(mu, sigma)

                with tf.name_scope('a_loss'):
                    log_prob = normal_dist.log_prob(self.a_his)
                    exp_v = log_prob * td
                    entropy = normal_dist.entropy()  # encourage exploration
                    self.exp_v = ENTROPY_BETA * entropy + exp_v
                    self.a_loss = tf.reduce_sum(-self.exp_v)

                with tf.name_scope('choose_a'):  # use local params to choose action
                    self.A = tf.clip_by_value(tf.squeeze(normal_dist.sample(1), axis=0), A_BOUND[0], A_BOUND[1])
                with tf.name_scope('local_grad'):
                    self.a_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/actor')
                    self.c_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/critic')
                    self.a_grads = tf.gradients(self.a_loss, self.a_params)
                    self.c_grads = tf.gradients(self.c_loss, self.c_params)

            with tf.name_scope('sync'):
                with tf.name_scope('pull'):
                    self.pull_a_params_op = [l_p.assign(g_p) for l_p, g_p in zip(self.a_params, globalAC.a_params)]
                    self.pull_c_params_op = [l_p.assign(g_p) for l_p, g_p in zip(self.c_params, globalAC.c_params)]
                if Train:
                    with tf.name_scope('push'):
                        self.update_a_op = OPT_A.apply_gradients(zip(self.a_grads, globalAC.a_params))
                        self.update_c_op = OPT_C.apply_gradients(zip(self.c_grads, globalAC.c_params))

    def _build_net(self ):
        w_init = tf.random_normal_initializer(0., .1)
        with tf.variable_scope('actor'):
            l_a_0 = tf.layers.dense(self.s, 512, tf.nn.relu6, kernel_initializer=w_init, name='la0')
            l_a_1 = tf.layers.dense(l_a_0, 512, tf.nn.relu6, kernel_initializer=w_init, name='la1')
            mu = tf.layers.dense(l_a_1, N_A, tf.nn.tanh, kernel_initializer=w_init, name='mu')
            sigma = tf.layers.dense(l_a_1, N_A, tf.nn.softplus, kernel_initializer=w_init, name='sigma')
        with tf.variable_scope('critic'):
            l_c0 = tf.layers.dense(self.s, 256, tf.nn.relu6, kernel_initializer=w_init, name='lc0')
            l_c1 = tf.layers.dense(l_c0, 256, tf.nn.relu6, kernel_initializer=w_init, name='lc1')
            v = tf.layers.dense(l_c1, 1, kernel_initializer=w_init, name='v')  # state value
        return mu, sigma, v

    def update_global(self, feed_dict):  # run by a local
        SESS.run([self.update_a_op, self.update_c_op], feed_dict)  # local grads applies to global net

    def pull_global(self):  # run by a local
        SESS.run([self.pull_a_params_op, self.pull_c_params_op])

    def choose_action(self, s):  # run by a local
        s = s[np.newaxis, :]
        return SESS.run(self.A, {self.s: s})[0]


class Worker(object):
    def __init__(self, name, globalAC):
        self.env = gym.make(GAME).unwrapped
        self.name = name
        self.AC = ACNet(name, globalAC)

    def work(self, saver):
        global GLOBAL_RUNNING_R, GLOBAL_EP
        total_step = 1
        buffer_s, buffer_a, buffer_r = [], [], []
        while not COORD.should_stop() and GLOBAL_EP < MAX_GLOBAL_EP:
            s = self.env.reset()
            ep_r = 0
            while True:
                if self.name == 'W_0':
                    self.env.render()
                a = self.AC.choose_action(s)
                #s_, r, done, info = self.env.step(a)
                #done = True if ep_r == MAX_EP_STEP - 1 else False
                r = 0 
                for _ in range(5):
                    if self.name == 'W_0':
                        self.env.render()
                    s_, r_, done, info = self.env.step(a)
                    if not done and r_ > 0:
                        r_ *= 25
                    #elif r_ > -99:
                    #    r_ = 0
                    elif r_ == -100:
                        r_ *= 10
                    r += r_
                    ep_r += r_
                    if done:
                        break
                

                ep_r += r
                buffer_s.append(s)
                buffer_a.append(a)
                buffer_r.append(r)

                if total_step % UPDATE_GLOBAL_ITER == 0 or done:   # update global and assign to local net
                    if done:
                        v_s_ = 0   # terminal
                    else:
                        v_s_ = SESS.run(self.AC.v, {self.AC.s: s_[np.newaxis, :]})[0, 0]
                    buffer_v_target = []
                    for r in buffer_r[::-1]:    # reverse buffer r
                        v_s_ = r + GAMMA * v_s_
                        buffer_v_target.append(v_s_)
                    buffer_v_target.reverse()

                    buffer_s, buffer_a, buffer_v_target = np.vstack(buffer_s), np.vstack(buffer_a), np.vstack(buffer_v_target)
                    feed_dict = {
                        self.AC.s: buffer_s,
                        self.AC.a_his: buffer_a,
                        self.AC.v_target: buffer_v_target,
                    }
                    self.AC.update_global(feed_dict)
                    buffer_s, buffer_a, buffer_r = [], [], []
                    self.AC.pull_global()

                s = s_
                total_step += 1
                if done:
                    if len(GLOBAL_RUNNING_R) == 0:  # record running episode reward
                        GLOBAL_RUNNING_R.append(ep_r)
                    else:
                        GLOBAL_RUNNING_R.append(0.9 * GLOBAL_RUNNING_R[-1] + 0.1 * ep_r)
                    print(
                        self.name,
                        "Ep:", GLOBAL_EP,
                        "| Ep_r: %i" % GLOBAL_RUNNING_R[-1],
                          )
                    GLOBAL_EP += 1
                    if GLOBAL_EP % CHECKPOINT_INTERVAL == 0 and Train:
                        saver.save(SESS, CHECKPOINT_SAVE_PATH, global_step = GLOBAL_EP)
                    break

if __name__ == "__main__":
    SESS = tf.Session()

    if Train:
        with tf.device("/cpu:0"):
            OPT_A = tf.train.RMSPropOptimizer(LR_A, name='RMSPropA')
            OPT_C = tf.train.RMSPropOptimizer(LR_C, name='RMSPropC')
            GLOBAL_AC = ACNet(GLOBAL_NET_SCOPE)  # we only need its params
            workers = []
            # Create worker
            for i in range(N_WORKERS):
                i_name = 'W_%i' % i   # worker name
                workers.append(Worker(i_name, GLOBAL_AC))

        COORD = tf.train.Coordinator()
        SESS.run(tf.global_variables_initializer())

        if OUTPUT_GRAPH:
            if os.path.exists(LOG_DIR):
                shutil.rmtree(LOG_DIR)
            tf.summary.FileWriter(LOG_DIR, SESS.graph)


        reuse_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=GLOBAL_NET_SCOPE )
        reuse_vars_dict = dict([(var.name, var) for var in reuse_vars])
        print(reuse_vars_dict)
        saver = tf.train.Saver(reuse_vars_dict)

        worker_threads = []
        for worker in workers:
            job = lambda: worker.work(saver)
            t = threading.Thread(target=job)
            t.start()
            worker_threads.append(t)
        COORD.join(worker_threads)

        plt.plot(np.arange(len(GLOBAL_RUNNING_R)), GLOBAL_RUNNING_R)
        plt.xlabel('step')
        plt.ylabel('Total moving reward')
        plt.show()
    else:
        episode = 1500
        with tf.device("/cpu:0"):
            GLOBAL_AC = ACNet(GLOBAL_NET_SCOPE)  # we only need its params
            LOCAL_AC = ACNet('ww', GLOBAL_AC)
            
        reuse_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=GLOBAL_NET_SCOPE )
        reuse_vars_dict = dict([(var.name, var) for var in reuse_vars])
        print(reuse_vars_dict)
        saver = tf.train.Saver(reuse_vars_dict)
        saver.restore(SESS, CHECKPOINT_SAVE_PATH + '-{}'.format(episode))
        
        
        env = gym.wrappers.Monitor(env, '/tmp/BipedalWalker-v2'+'-{}'.format(episode),force=True)
        LOCAL_AC.pull_global()
        for i_episode in range(20):
            observation = env.reset()
            done = False
            t = 0
            while not done:
                t += 1
                env.render()
                #print(observation)
                action = LOCAL_AC.choose_action(observation)  # get the action and next rnn state
                observation, reward, done, info = env.step(action)
                if done:
                    print("Episode finished after {} timesteps".format(t+1))
                    t = 0
                    break




[2017-06-11 02:45:33,143] Making new env: BipedalWalker-v2


{'Global_Net/actor/la0/kernel:0': <tf.Variable 'Global_Net/actor/la0/kernel:0' shape=(24, 512) dtype=float32_ref>, 'Global_Net/actor/la0/bias:0': <tf.Variable 'Global_Net/actor/la0/bias:0' shape=(512,) dtype=float32_ref>, 'Global_Net/actor/la1/kernel:0': <tf.Variable 'Global_Net/actor/la1/kernel:0' shape=(512, 512) dtype=float32_ref>, 'Global_Net/actor/la1/bias:0': <tf.Variable 'Global_Net/actor/la1/bias:0' shape=(512,) dtype=float32_ref>, 'Global_Net/actor/mu/kernel:0': <tf.Variable 'Global_Net/actor/mu/kernel:0' shape=(512, 4) dtype=float32_ref>, 'Global_Net/actor/mu/bias:0': <tf.Variable 'Global_Net/actor/mu/bias:0' shape=(4,) dtype=float32_ref>, 'Global_Net/actor/sigma/kernel:0': <tf.Variable 'Global_Net/actor/sigma/kernel:0' shape=(512, 4) dtype=float32_ref>, 'Global_Net/actor/sigma/bias:0': <tf.Variable 'Global_Net/actor/sigma/bias:0' shape=(4,) dtype=float32_ref>, 'Global_Net/critic/lc0/kernel:0': <tf.Variable 'Global_Net/critic/lc0/kernel:0' shape=(24, 256) dtype=float32_ref>, 

[2017-06-11 02:45:34,428] Restoring parameters from /tmp/a3c_NN_sum_r10.ckpt-1500
[2017-06-11 02:45:34,471] Creating monitor directory /tmp/BipedalWalker-v2-1500
[2017-06-11 02:45:34,492] Starting new video recorder writing to /tmp/BipedalWalker-v2-1500/openaigym.video.0.17206.video000000.mp4
[2017-06-11 02:46:01,732] Starting new video recorder writing to /tmp/BipedalWalker-v2-1500/openaigym.video.0.17206.video000001.mp4


Episode finished after 378 timesteps
Episode finished after 640 timesteps
Episode finished after 196 timesteps
Episode finished after 114 timesteps
Episode finished after 1222 timesteps
Episode finished after 225 timesteps


KeyboardInterrupt: 